In [2]:
#hide
from integrators import *

# Integrators

> Integrators integrate your information in the pod. They import your data from external services (gmail, whatsapp, icloud, facebook etc.), enrich your data with indexers (face recognition, spam detection, duplicate photo detection), and execute actions (sending mails, automatically share selected photo's with your family).

Integrators for memri have a single repo per language. This repo contains all python integrators. This repo is build with [nbdev](https://github.com/fastai/nbdev). 

## Install

`pip install -e integrators`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2